In [1]:
%matplotlib inline

In [17]:
from __future__ import division

import sys
import time
import logging

import numpy as np
from scipy.io import loadmat
from os import path

# change this to the path of mushu if you don't have it in your
# PYTHONPATH already
sys.path.append('../../mushu')
sys.path.append('../')

import libmushu
from libmushu.driver.replayamp import ReplayAmp
from wyrm.types import RingBuffer
import wyrm.processing as proc
from wyrm import io
from wyrm.types import Data, BlockBuffer, RingBuffer
from pandas import read_csv      

#Classifiers
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import matplotlib
import matplotlib.pyplot as plt

from sklearn.externals import joblib
from os import listdir
from os.path import isfile, join

In [18]:
logging.basicConfig(format='%(relativeCreated)10.0f %(threadName)-10s %(name)-10s %(levelname)8s %(message)s', level=logging.NOTSET)
logger = logging.getLogger(__name__)

In [19]:
# replay the experiment in real time?
REALTIME = False


TRAIN_DATA_A = '../BCI_Comp_III_Wads_2004/data/Subject_A_Train.mat'
TEST_DATA_A = '../BCI_Comp_III_Wads_2004/data/Subject_A_Test.mat'

TRAIN_DATA_B = '../BCI_Comp_III_Wads_2004/data/Subject_B_Train.mat'
TEST_DATA_B = '../BCI_Comp_III_Wads_2004/data/Subject_B_Test.mat'


CHANNEL_DATA = '../BCI_Comp_III_Wads_2004/data/eloc64.txt'

TRUE_LABELS_TEST = "WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU"

STIMULUS_CODE = {
    # cols from left to right
    1 : "agmsy5",
    2 : "bhntz6",
    3 : "ciou17",
    4 : "djpv28",
    5 : "ekqw39",
    6 : "flrx4_",
    # rows from top to bottom
    7 : "abcdef",
    8 : "ghijkl",
    9 : "mnopqr",
    10: "stuvwx",
    11: "yz1234",
    12: "56789_"
}

MARKER_DEF_TRAIN = {'target': ['target'], 'nontarget': ['nontarget']}
MARKER_DEF_TEST = {i : [i] for i in STIMULUS_CODE.values()}

SEG_IVAL = [0, 800]

In [20]:
def load_bci_TESTdata(filename):
    """Load the BCI Competition III Data Set 2.
    This method loads the data set and converts it into Wyrm's ``Data``
    format. Before you use it, you have to download the data set in
    Matlab format and unpack it. The directory with the extracted files
    must contain the ``Subject_*.mat``- and the ``eloc64.txt`` files.
    .. note::
        If you need the true labels of the test sets, you'll have to
        download them separately from
        http://bbci.de/competition/iii/results/index.html#labels
    Parameters
    ----------
    filename : str
        The path to the matlab file to load
    Returns
    -------
    cnt : continuous `Data` object
    Examples
    --------
    >>> dat = load_bcicomp3_ds2('/home/foo/data/Subject_A_Train.mat')
    """
    STIMULUS_CODE = {
        0 : "blankMatrix",
        # cols from left to right
        1 : "agmsy5",
        2 : "bhntz6",
        3 : "ciou17",
        4 : "djpv28",
        5 : "ekqw39",
        6 : "flrx4_",
        # rows from top to bottom
        7 : "abcdef",
        8 : "ghijkl",
        9 : "mnopqr",
        10: "stuvwx",
        11: "yz1234",
        12: "56789_"
        }

    # load the matlab data
    data_mat = loadmat(filename)
    # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filename), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2
    # The signal is recorded with 64 channels, bandpass filtered
    # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
    # Samples x Channels
    data = data_mat['Signal'][60:,:,:]
    data = data.astype('double')
#     print('data: ',data.shape)
    # For each sample: 1 if a row/colum was flashed, 0 otherwise
    flashing = data_mat['Flashing'][60:,:]
    flashing = flashing.reshape(-1)
#     print('flashing: ',flashing.shape)
    #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
    ##Creates an array where only the initial intensifications of each series appear
    tmp = []
    for i, _ in enumerate(flashing):
        if i == 0:
            tmp.append(flashing[i])
            continue
        if flashing[i] == flashing[i-1] == 1:
            tmp.append(0)
            continue
        tmp.append(flashing[i])
    flashing = np.array(tmp)
    # For each sample: 0 when no row/colum was intensified,
    # 1..6 for intensified columns, 7..12 for intensified rows
    stimulus_code = data_mat['StimulusCode'][60:,:].reshape(-1)
#     print('stim_code: ', stimulus_code.shape)
    stimulus_code = stimulus_code[flashing == 1]
    # 0 if no row/col was intensified or the intensified did not contain
    # the target character, 1 otherwise
    stimulus_type = data_mat.get('StimulusType', np.array([]))[60:,:].reshape(-1)
#     print('stim_type: ', stimulus_type.shape)
    # The target characters
    target_chars = data_m.get('TargetChar')[0][60:]
#     target_chars = data_mat.get('TargetChar', np.array([])).reshape(-1)
    fs = 240
    data = data.reshape(-1, 64)
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    #stimulus_code = zip([t for t, _ in flashing], [STIMULUS_CODE[i] for i in stimulus_code])
    #Raises error "TypeError: '<' not supported between instances of 'tuple' and 'list'" when calling sort() 
    #stimulus_code =[[t for t,_ in flashing], [STIMULUS_CODE[i] for i in stimulus_code]]
    #print(type(stimulus_code), type(flashing), type(targets), type(nontargets))
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
    return dat


In [21]:
def load_bci_TRAINdata(filename):
    """Load the BCI Competition III Data Set 2.
    This method loads the data set and converts it into Wyrm's ``Data``
    format. Before you use it, you have to download the data set in
    Matlab format and unpack it. The directory with the extracted files
    must contain the ``Subject_*.mat``- and the ``eloc64.txt`` files.
    .. note::
        If you need the true labels of the test sets, you'll have to
        download them separately from
        http://bbci.de/competition/iii/results/index.html#labels
    Parameters
    ----------
    filename : str
        The path to the matlab file to load
    Returns
    -------
    cnt : continuous `Data` object
    Examples
    --------
    >>> dat = load_bcicomp3_ds2('/home/foo/data/Subject_A_Train.mat')
    """
    STIMULUS_CODE = {
        0 : "blankMatrix",
        # cols from left to right
        1 : "agmsy5",
        2 : "bhntz6",
        3 : "ciou17",
        4 : "djpv28",
        5 : "ekqw39",
        6 : "flrx4_",
        # rows from top to bottom
        7 : "abcdef",
        8 : "ghijkl",
        9 : "mnopqr",
        10: "stuvwx",
        11: "yz1234",
        12: "56789_"
        }

    # load the matlab data
    data_mat = loadmat(filename)
    # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filename), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2
    # The signal is recorded with 64 channels, bandpass filtered
    # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
    # Samples x Channels
    data = data_mat['Signal'][:60,:,:]
    data = data.astype('double')
#     print('data: ',data.shape)
    # For each sample: 1 if a row/colum was flashed, 0 otherwise
    flashing = data_mat['Flashing'][:60,:]
    flashing = flashing.reshape(-1)
#     print('flashing: ',flashing.shape)
    #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
    ##Creates an array where only the initial intensifications of each series appear
    tmp = []
    for i, _ in enumerate(flashing):
        if i == 0:
            tmp.append(flashing[i])
            continue
        if flashing[i] == flashing[i-1] == 1:
            tmp.append(0)
            continue
        tmp.append(flashing[i])
    flashing = np.array(tmp)
    # For each sample: 0 when no row/colum was intensified,
    # 1..6 for intensified columns, 7..12 for intensified rows
    stimulus_code = data_mat['StimulusCode'][:60,:].reshape(-1)
#     print('stim_code: ', stimulus_code.shape)
    stimulus_code = stimulus_code[flashing == 1]
    # 0 if no row/col was intensified or the intensified did not contain
    # the target character, 1 otherwise
    stimulus_type = data_mat.get('StimulusType', np.array([]))[:60,:].reshape(-1)
#     print('stim_type: ', stimulus_type.shape)
    # The target characters
    target_chars = data_m.get('TargetChar')[0][:60]
#     target_chars = data_mat.get('TargetChar', np.array([])).reshape(-1)
    
    fs = 240
    data = data.reshape(-1, 64)
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    #stimulus_code = zip([t for t, _ in flashing], [STIMULUS_CODE[i] for i in stimulus_code])
    #Raises error "TypeError: '<' not supported between instances of 'tuple' and 'list'" when calling sort() 
    #stimulus_code =[[t for t,_ in flashing], [STIMULUS_CODE[i] for i in stimulus_code]]
    #print(type(stimulus_code), type(flashing), type(targets), type(nontargets))
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
    return dat


In [22]:
def train(filename):
    cnt = load_bci_TRAINdata(filename)

    fs_n = cnt.fs / 2

    b, a = proc.signal.butter(5, [10 / fs_n], btype='low')
    cnt = proc.lfilter(cnt, b, a)

#     b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
#     cnt = proc.lfilter(cnt, b, a)

    cnt = proc.subsample(cnt, 20)

    epo = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL)

    #from wyrm import plot
    #plot.plot_spatio_temporal_r2_values(proc.sort_channels(epo))
    #print JUMPING_MEANS_IVALS
    #plot.plt.show()

    fv = proc.create_feature_vectors(epo)
    X = fv.data
    y = fv.axes[0]
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, train_size = 0.7)

    #     clf = proc.lda_train(fv)
    clf = LinearDiscriminantAnalysis(solver='eigen')
    clf.fit(X,y)
    return clf

In [23]:


from __future__ import division

import time

import numpy as np

from libmushu.amplifier import Amplifier


class RReeplayAmp(Amplifier):

    def configure(self, data, marker, channels, fs, realtime=True, blocksize_ms=None, blocksize_samples=None):
        """

        Parameters
        ----------
        data
        marker
        channels
        fs
        realtime
        blocksize_ms : float
            blocksize in milliseconds
        blocksize_samples : int
            blocksize in samples

        Raises
        ------
        TypeError :
            if ``blocksize_ms`` and ``blocksize_s`` are given.

        """
        if [blocksize_ms, blocksize_samples].count(None) != 1:
            raise TypeError("blocksize_ms and blocksize_samples are mutually exclusive.")

        self.data = data
        # slow python
        self.marker = marker
        # fast numpy
        self.marker_ts = np.array([ts for ts, s in marker]) #marker time_sample
        self.marker_s = np.array([s for ts, s in marker]) #marker sample
        self.channels = channels
        self.fs = fs
        self.realtime = realtime

        if blocksize_ms:
            samples = fs * (blocksize_ms / 1000)
            if not samples.is_integer():
                raise ValueError("Resulting blocksize is not integer, please fix the blocksize_ms.")
            self.samples = samples
        if blocksize_samples:
            self.samples = blocksize_samples

    def start(self):
        self.last_sample_time = time.time()
        self.pos = 0

    def stop(self):
        pass

    def get_data(self):
        """

        Returns
        -------
        chunk, markers: Markers is time in ms since relative to the
        first sample of that block.

        """
        if self.realtime:
            elapsed = time.time() - self.last_sample_time
            blocks = (self.fs * elapsed) // self.samples
            samples = int(blocks * self.samples)
            
        else:
            samples = self.samples
      
        elapsed = samples / self.fs
        self.last_sample_time += elapsed
        # data
        chunk = self.data[self.pos:int(self.pos+samples)]
        #self.data = self.data[samples:]
        # markers

        # slow python version
        #markers = [x for x in self.marker if x[0] < elapsed * 1000]
        #self.marker = [x for x in self.marker if x[0] >= elapsed * 1000]
        #self.marker = [[x[0] - elapsed * 1000, x[1]] for x in self.marker]

        # fast numpy version
        mask = self.marker_ts < (elapsed * 1000) 
        markers = list(zip(self.marker_ts[mask], self.marker_s[mask]))
        self.marker_ts = self.marker_ts[~mask]
        self.marker_s = self.marker_s[~mask]
        self.marker_ts -= elapsed * 1000

        self.pos += samples
        return chunk, markers

    def get_channels(self):
        return self.channels

    def get_sampling_frequency(self):
        return self.fs

    @staticmethod
    def is_available():
        return True


In [38]:
def online_experiment(amp, cfy, subject):
    amp_fs = amp.get_sampling_frequency()
    amp_channels = amp.get_channels()
    
    file = open('sim_training_LDA'+subject+'.txt', 'w')
    file.write('LDA Online simulation'+subject)
    
    # csv results file
    filename = 'results/'+subject+'/Results_LDA_'+subject+'.csv'
    columnNames = 'Character,Trial,Correct/12,isCorrect_ch_pred,recall_acum,precision_acum,accuracy'

    resultsFile = open(filename, 'w')
    resultsFile.write(columnNames)

    chfilename = 'results/'+subject+'/CharacterResults_LDA_'+subject+'.csv'
    chcolumnNames = 'Character,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5, Trial 6,\
    Trial 7,Trial 8,Trial 9,Trial 10,Trial 11,Trial 12,Trial 13,Trial 14,Trial 15,Trial 16'
    chResultsFile = open(chfilename, 'w')
    chResultsFile.write(chcolumnNames) 
    

    #buf = BlockBuffer(4)
    rb = RingBuffer(5000)

    fn = amp_fs / 2
    b_low, a_low = proc.signal.butter(5, [10 / fn], btype='low')
#     b_high, a_high = proc.signal.butter(5, [.4 / fn], btype='high')

    zi_low = proc.lfilter_zi(b_low, a_low, len(amp_channels))
#     zi_high = proc.lfilter_zi(b_high, a_high, len(amp_channels))

    amp.start()
    markers_processed = 0
#     current_letter_idx = 0
#     current_letter = TRUE_LABELS[current_letter_idx].lower()

    letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
    endresult = []
    file.write('\nCharacter #'+str(len(endresult)+1))

   # t0 = time.time()
    #data, markers = amp.get_data()
    
    y_true = np.array([])
    y_pred = np.array([])
    
    correct_letter = False #boolean to know if the letter has already been predicted
    
    while True:
        t0 = time.time()

        # get fresh data from the amp
        data, markers = amp.get_data()
        if len(data) == 0:
            continue

        # we should rather wait for a specific end-of-experiment marker
        if len(data) == 0:
            break

        # convert to cnt
        cnt = io.convert_mushu_data(data, markers, amp_fs, amp_channels)
        
        ## enter the block buffer
        #buf.append(cnt)
        #cnt = buf.get()
        #if not cnt:
        #    continue

        # low-pass and subsample
        cnt, zi_low = proc.lfilter(cnt, b_low, a_low, zi=zi_low)
#         cnt, zi_high = proc.lfilter(cnt, b_high, a_high, zi=zi_high)

        cnt = proc.subsample(cnt, 20)

        newsamples = cnt.data.shape[0]

        # enter the ringbuffer
        rb.append(cnt)
        cnt = rb.get()

        # segment
        epo = proc.segment_dat(cnt, MARKER_DEF_TEST, SEG_IVAL, newsamples=newsamples)
        epo_train = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL, newsamples=newsamples)
        if not epo: # If not enough data for 800 ms
            continue

        fv = proc.create_feature_vectors(epo)
        fv_train = proc.create_feature_vectors(epo_train)
        
        logger.debug('Markers processed: ')
        logger.debug(markers_processed)
        
        lda_out = cfy.predict_proba(fv.data)[:,1]
        y_true = np.append(y_true, fv_train.axes[0])
        
        y_pred_now = cfy.predict(fv.data) 
        y_pred = np.append(y_pred, y_pred_now)
        
        y_correct = y_pred_now == fv_train.axes[0]
        print('Predicho: ', fv_train.axes[0],'\tVerdadero: ', y_pred_now, y_correct )

#         lda_out = proc.lda_apply(fv, cfy)
        # class_names = letters in matrix row/column
        markers = [fv.class_names[cls_idx] for cls_idx in fv.axes[0]]
        result = zip(markers, lda_out)
        
        
        # Accumulated metrics
        recall = metrics.recall_score(y_true, y_pred)
        precision = metrics.precision_score(y_true, y_pred)
        accuracy = metrics.accuracy_score(y_true, y_pred)
        f1 = metrics.f1_score(y_true, y_pred)
        print('\nP300 classification metrics...')
        print('recall: ', recall, '\tprecision: ', precision, '\tf1: ', f1, '\taccuracy: ', accuracy)
        for s, score in result: #letters in one row/column
            for letter in s:
                print(letter)
                letter_prob[letter] += score
            if (markers_processed%12 == 0):
               
              
                trues = '\n\nTrial #' + str(int(markers_processed/12)+1) + ':\t' + str(np.count_nonzero(y_pred[-12:] == y_true[-12:])) + '/12 correct P300 predictions'
                file.write(trues)
                res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
                file.write('\nAccumulated metrics: ')
                file.write(res)
                current_letter = sorted(letter_prob.items(), key=lambda x: x[1])[-1][0]
                if (current_letter.upper() == TRUE_LABELS[len(endresult)]) & (not correct_letter):
                    n_trial = int(markers_processed/12)+1
                    file.write('\n\n~~~~~Character #{} ({}) predicted in {} trials'.format(len(endresult)+1,
                                                                                  current_letter, n_trial))
                    correct_letter = True
#                 if (markers_processed != 180) & (not correct_letter):
#                     if(current_letter.upper() == TRUE_LABELS[len(endresult)]):
#                          file.write('\n\nCharacter #{} ({}) predicted in {} trials'.format(len(endresult),
#                                                                                           current_letter, int(markers_processed/12)))

                # Ch_#, Trial_#, isCorrect_ch_pred, correct/12, recall_now, precision_now, 
                #recall, precision, accuracy
                resultsFile.write('\n'+str(len(endresult)+1)+','+str(int(markers_processed/12)+1)+','
                                  +str(np.count_nonzero(y_pred[-12:] == y_true[-12:]))+','+str(correct_letter)+
                                  ','+str(recall)+','+str(precision)+','+str(accuracy))

                if markers_processed == 180: # 12 trials x 15 repetitions = 180
                    endresult.append(sorted(letter_prob.items(), key=lambda x: x[1])[-1][0])
                    # Set all probabilities to zero
                    letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
                    markers_processed = -1
                    if not correct_letter:
                        file.write('\n\nFailed to predict character #{}'.format(len(endresult)))
    #                         file.write('\nPredicted {} instead of {}.'.format(current_letter.upper(), TRUE_LABELS[len(endresult)]))
    #                         res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
    #                         file.write(res)
                    n_ch = len(endresult)
                    # 0: character not predicted in trial, 1: character correctly predicted
                    if correct_letter:
                        trial_results = ('0,'*(n_trial-1)+(16-n_trial+1)*'1,')[:-1]
                    else:
                        trial_results = ('0,'*16)[:-1]
                    
                    chResultsFile.write('\n'+str(n_ch)+','+trial_results)
                    correct_letter = False        
                    file.write('\n\nCharacter #'+str(n_ch+1))
        

            markers_processed += 1

                
#             if (markers_processed%12 == 0) & (markers_processed != 180) & (not correct_letter):
#                 current_letter = sorted(letter_prob.items(), key=lambda x: x[1])[-1][0]
#                 if(current_letter.upper() == TRUE_LABELS[len(endresult)]):
#                     file.write('\n\nCharacter #{} ({}) predicted in {} trials'.format(len(endresult),
#                                                                                   current_letter, int(markers_processed/12)))
#                     res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
#                     file.write(res)
#                     correct_letter = True
           
#             elif markers_processed == 180: # 12 trials x 15 repetitions = 180
#                 endresult.append(sorted(letter_prob.items(), key=lambda x: x[1])[-1][0])
#                 # Set all probabilities to zero
#                 letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
#                 markers_processed = 0
#                 if not correct_letter:
#                     file.write('\n\nFailed to predict character #{} in 15 trials'.format(len(endresult)))
#                     file.write('\nPredicted {} instead of {}.'.format(current_letter.upper(), TRUE_LABELS[len(endresult)]))
#                     res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
#                     file.write(res)
                    
#                 correct_letter = False
# #                 current_letter_idx += 1
# #                 print('letter idx', current_letter_idx)
# #                 current_letter = TRUE_LABELS[current_letter_idx].lower()
#             markers_processed += 1
#         print(letter_prob)
#         logger.debug("".join([i[0] for i in sorted(letter_prob.items(), key=lambda x: x[1], reverse=True)]).replace(current_letter, " %s " % current_letter))

        # calculate the current accuracy
        if len(endresult) > 0:
            logger.debug(TRUE_LABELS)
            logger.debug("".join(endresult))
            acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
            print('Character prediction metrics...')
            print("Current accuracy:", acc * 100)
            print('{}/{} characters predicted'.format(len(endresult), len(TRUE_LABELS)))
        if len(endresult) == len(TRUE_LABELS)-1:
            break
        #logger.debug("Result: %s" % result)
        print('time',1000 * (time.time() - t0))

    acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
    print ("Accuracy:", acc * 100)
    file.close()
    resultsFile.close()

    amp.stop()


In [25]:
subject = 'A'
if subject == 'A':
    TRAIN_DATA = TRAIN_DATA_A
elif subject == 'B':
    TRAIN_DATA = TRAIN_DATA_B
data_m = loadmat(TRAIN_DATA)
target_chars = data_m.get('TargetChar')[0][60:]
TRUE_LABELS = target_chars

In [26]:
TRUE_LABELS

'W_EKTLBWXEPOUIKZERYOOTHQI'

# 1. Training (off-line)

In [12]:
clf = train(TRAIN_DATA)

In [27]:
filename = 'Trained Classifiers/'+subject+'/LDA_'+subject+'.pkl'

In [13]:
joblib.dump(clf, filename)

['Trained Classifiers/B/LDA_B.pkl']

In [28]:
clf = joblib.load(filename)

# 2. Online experiment

In [34]:
bbuffer = BlockBuffer(12)
cnt = load_bci_TESTdata(TRAIN_DATA)

In [35]:
amp = RReeplayAmp()
amp.configure(data=cnt.data, marker=cnt.markers, channels=cnt.axes[-1], fs=cnt.fs,
              realtime=False, blocksize_samples=12)

In [37]:
online_experiment(amp, clf, subject)

DEBUG:__main__:Markers processed: 
DEBUG:__main__:0
DEBUG:__main__:Markers processed: 
DEBUG:__main__:1
DEBUG:__main__:Markers processed: 
DEBUG:__main__:2
DEBUG:__main__:Markers processed: 
DEBUG:__main__:3
DEBUG:__main__:Markers processed: 
DEBUG:__main__:4
DEBUG:__main__:Markers processed: 
DEBUG:__main__:5
DEBUG:__main__:Markers processed: 
DEBUG:__main__:6
DEBUG:__main__:Markers processed: 
DEBUG:__main__:7


Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
s,score  mnopqr 0.10302175701100938
m
n
o
p
q
r
time 7.97724723815918
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
s,score  yz1234 0.0025857618866221167
y
z
1
2
3
4
time 9.974241256713867
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
s,score  agmsy5 5.793807227486886e-08
a
g
m
s
y
5
time 8.97669792175293
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
s,score  ghijkl 2.512280336744239e-09
g
h
i
j
k
l
time 9.97304916381836
Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification metrics...
recall:  1.0 	precision:  1.0 	f1:  1.0 	accuracy:  1.0
s,score  ekqw39 0.999592484578395
e
k
q
w
3
9
time 12.966394424438477
Pre

DEBUG:__main__:Markers processed: 
DEBUG:__main__:8
DEBUG:__main__:Markers processed: 
DEBUG:__main__:9
DEBUG:__main__:Markers processed: 
DEBUG:__main__:10
DEBUG:__main__:Markers processed: 
DEBUG:__main__:11
DEBUG:__main__:Markers processed: 
DEBUG:__main__:12
DEBUG:__main__:Markers processed: 


Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5 	precision:  1.0 	f1:  0.6666666666666666 	accuracy:  0.8888888888888888
s,score  abcdef 0.002155002000138682
a
b
c
d
e
f
time 18.949508666992188
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5 	precision:  1.0 	f1:  0.6666666666666666 	accuracy:  0.9
s,score  ciou17 3.217726962681244e-09
c
i
o
u
1
7
time 15.95759391784668
Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.5 	precision:  0.5 	f1:  0.5 	accuracy:  0.8181818181818182
s,score  djpv28 0.999999901682077
d
j
p
v
2
8
time 14.960765838623047
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5 	precision:  0.5 	f1:  0.5 	accuracy:  0.8333333333333334
s,score  djpv28 6.456200362264764e-08
d
j
p
v
2
8
time 16.95394515991211
Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.666666666

DEBUG:__main__:13
DEBUG:__main__:Markers processed: 
DEBUG:__main__:14
DEBUG:__main__:Markers processed: 
DEBUG:__main__:15
DEBUG:__main__:Markers processed: 
DEBUG:__main__:16
DEBUG:__main__:Markers processed: 
DEBUG:__main__:17


...

DEBUG:__main__:Markers processed: 
DEBUG:__main__:121
DEBUG:__main__:Markers processed: 
DEBUG:__main__:122
DEBUG:__main__:Markers processed: 
DEBUG:__main__:123
DEBUG:__main__:Markers processed: 
DEBUG:__main__:124


Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6 	precision:  0.375 	f1:  0.4615384615384615 	accuracy:  0.7704918032786885
s,score  yz1234 0.026090099570394234
y
z
1
2
3
4
time 20.944595336914062
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6 	precision:  0.375 	f1:  0.4615384615384615 	accuracy:  0.7723577235772358
s,score  flrx4_ 0.005887937257453106
f
l
r
x
4
_
time 24.932861328125
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6 	precision:  0.375 	f1:  0.4615384615384615 	accuracy:  0.7741935483870968
s,score  56789_ 3.1959534496729412e-09
5
6
7
8
9
_
time 19.946575164794922
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6 	precision:  0.375 	f1:  0.4615384615384615 	accuracy:  0.776
s,score  djpv28 0.0016649446609614277
d
j
p
v
2
8
time 27.925491333007812


DEBUG:__main__:Markers processed: 
DEBUG:__main__:125
DEBUG:__main__:Markers processed: 
DEBUG:__main__:126
DEBUG:__main__:Markers processed: 
DEBUG:__main__:127
DEBUG:__main__:Markers processed: 
DEBUG:__main__:128


Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification metrics...
recall:  0.6190476190476191 	precision:  0.3939393939393939 	f1:  0.48148148148148145 	accuracy:  0.7777777777777778
s,score  stuvwx 0.9999268489420557
s
t
u
v
w
x
time 19.945621490478516
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6190476190476191 	precision:  0.3939393939393939 	f1:  0.48148148148148145 	accuracy:  0.7795275590551181
s,score  abcdef 0.022263890259056705
a
b
c
d
e
f
time 15.960454940795898
Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification metrics...
recall:  0.6363636363636364 	precision:  0.4117647058823529 	f1:  0.5 	accuracy:  0.78125
s,score  ekqw39 0.9999294215938791
e
k
q
w
3
9
time 23.940324783325195


KeyboardInterrupt: 

In [1]:
import pandas as pd
names = ['Character #', 'Trial #', 'Correct/12', 'Correct Character Prediction']
X = pd.read_csv('Results_sim_LDA.txt', sep=",", skiprows = 0, names = names)

In [2]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(X)

     Character #  Trial #  Correct/12  Correct Character Prediction
0              1        1           1                         False
1              1        2           9                         False
2              1        3           8                         False
3              1        4          10                         False
4              1        5           7                         False
5              1        6           8                         False
6              1        7           9                         False
7              1        8           8                         False
8              1        9           6                         False
9              1       10           7                         False
10             1       11           7                         False
11             1       12           6                         False
12             1       13           7                         False
13             1       14           9           

In [3]:
X

,Character #,Trial #,Correct/12,Correct Character Prediction
0,1,1,1,False
1,1,2,9,False
2,1,3,8,False
3,1,4,10,False
4,1,5,7,False
5,1,6,8,False
6,1,7,9,False
7,1,8,8,False
8,1,9,6,False
9,1,10,7,False


In [10]:
X_correctAvg = X.drop(columns=['Character #', 'Trial #', 'Correct Character Prediction'])
X_correctAvg = X_correctAvg.groupby(np.arange(len(X))//16).mean()

In [12]:
X_correctAvg

,Correct/12
0,7.3750
1,6.7500
2,8.5000
3,9.0000
4,8.6250
5,7.9375
6,7.8750
7,7.5625
8,8.0625
9,8.3125


In [20]:
X2 = X.drop(columns=['Character #', 'Trial #'])
X2 = X2.groupby(np.arange(len(X2))//16)



In [16]:
X

,Character #,Trial #,Correct/12,Correct Character Prediction,new_index
0,1,1,1,False,NaN
1,1,2,9,False,NaN
2,1,3,8,False,NaN
3,1,4,10,False,NaN
4,1,5,7,False,NaN
5,1,6,8,False,NaN
6,1,7,9,False,NaN
7,1,8,8,False,NaN
8,1,9,6,False,NaN
9,1,10,7,False,NaN


## Sujeto C

In [159]:
def load_bci_TRAINdata2(filenames):

    STIMULUS_CODE = {
            0 : "blankMatrix",
            # cols from left to right
            1 : "agmsy5",
            2 : "bhntz6",
            3 : "ciou17",
            4 : "djpv28",
            5 : "ekqw39",
            6 : "flrx4_",
            # rows from top to bottom
            7 : "abcdef",
            8 : "ghijkl",
            9 : "mnopqr",
            10: "stuvwx",
            11: "yz1234",
            12: "56789_"
            }


    # # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filenames[0]), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2

    data_mat0 = loadmat(filenames[0])    
    data = data_mat0['signal']
    flashing = data_mat0['Flashing'].reshape(-1)
    stimulus_code = data_mat0['StimulusCode'].reshape(-1)
    stimulus_type = data_mat0.get('StimulusType', np.array([])).reshape(-1)
    sequence_phase = data_mat0['PhaseInSequence']

    #print(data.shape, flashing.shape, stimulus_code.shape, stimulus_type.shape)

    for i, filename in enumerate(filenames):
        if i == 0:
            continue
        # load the matlab data
        data_mat = loadmat(filename)
        # The signal is recorded with 64 channels, bandpass filtered
        # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
        # Samples x Channels
        data_temp = data_mat['signal']
        data = np.append(data, data_temp).reshape(-1, 64)
        data = data.astype('double')

        # For each sample: 1 if a row/colum was flashed, 0 otherwise
        flashing = np.append(flashing, data_mat['Flashing'].reshape(-1))
        #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
        ##Creates an array where only the initial intensifications of each series appear
        tmp = []
        for i, _ in enumerate(flashing):
            if i == 0:
                tmp.append(flashing[i])
                continue
            if flashing[i] == flashing[i-1] == 1:
                tmp.append(0)
                continue
            tmp.append(flashing[i])
        flashing = np.array(tmp)

        # For each sample: 0 when no row/colum was intensified,
        # 1..6 for intensified columns, 7..12 for intensified rows
        stimulus_code = np.append(stimulus_code, data_mat['StimulusCode'].reshape(-1))

        # 0 if no row/col was intensified or the intensified did not contain
        # the target character, 1 otherwise
        stimulus_type = np.append(stimulus_type, data_mat.get('StimulusType', np.array([]).reshape(-1)))
        sequence_phase = np.append(sequence_phase, data_mat['PhaseInSequence'])
  
    # Erase last 12 samples in order to reshape to (42,-1)
    flashing = flashing.reshape(-1)[:-12].reshape(42,-1)[:30,:].reshape(-1)
    stimulus_code = stimulus_code.reshape(-1)[:-12].reshape(42,-1)[:30,:].reshape(-1)
    stimulus_type = stimulus_type.reshape(-1)[:-12].reshape(42,-1)[:30,:].reshape(-1)
    
    # Erase last 768 samples in order to reshape to (42,-1,64 )
    data = data.reshape(-1)[:-768].reshape(42,-1,64)
    data = data[:30,:,:] # Get first 20 characters for training data 
    data = data.reshape(-1, 64)
    fs = 240
    stimulus_code = stimulus_code[flashing == 1]
 
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
   # dat.sequence_phase = sequence_phase
    return dat


In [158]:
def load_bci_TESTdata2(filenames):

    STIMULUS_CODE = {
            0 : "blankMatrix",
            # cols from left to right
            1 : "agmsy5",
            2 : "bhntz6",
            3 : "ciou17",
            4 : "djpv28",
            5 : "ekqw39",
            6 : "flrx4_",
            # rows from top to bottom
            7 : "abcdef",
            8 : "ghijkl",
            9 : "mnopqr",
            10: "stuvwx",
            11: "yz1234",
            12: "56789_"
            }


    # # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filenames[0]), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2

    data_mat0 = loadmat(filenames[0])    
    data = data_mat0['signal']
    flashing = data_mat0['Flashing'].reshape(-1)
    stimulus_code = data_mat0['StimulusCode'].reshape(-1)
    stimulus_type = data_mat0.get('StimulusType', np.array([])).reshape(-1)
    sequence_phase = data_mat0['PhaseInSequence']

    #print(data.shape, flashing.shape, stimulus_code.shape, stimulus_type.shape)

    for i, filename in enumerate(filenames):
        if i == 0:
            continue
        # load the matlab data
        data_mat = loadmat(filename)
        # The signal is recorded with 64 channels, bandpass filtered
        # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
        # Samples x Channels
        data_temp = data_mat['signal']
        data = np.append(data, data_temp).reshape(-1, 64)
        data = data.astype('double')

        # For each sample: 1 if a row/colum was flashed, 0 otherwise
        flashing = np.append(flashing, data_mat['Flashing'].reshape(-1))
        #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
        ##Creates an array where only the initial intensifications of each series appear
        tmp = []
        for i, _ in enumerate(flashing):
            if i == 0:
                tmp.append(flashing[i])
                continue
            if flashing[i] == flashing[i-1] == 1:
                tmp.append(0)
                continue
            tmp.append(flashing[i])
        flashing = np.array(tmp)

        # For each sample: 0 when no row/colum was intensified,
        # 1..6 for intensified columns, 7..12 for intensified rows
        stimulus_code = np.append(stimulus_code, data_mat['StimulusCode'].reshape(-1))

        # 0 if no row/col was intensified or the intensified did not contain
        # the target character, 1 otherwise
        stimulus_type = np.append(stimulus_type, data_mat.get('StimulusType', np.array([]).reshape(-1)))
        sequence_phase = np.append(sequence_phase, data_mat['PhaseInSequence'])
  
    # Erase last 12 samples in order to reshape to (42,-1)
    flashing = flashing.reshape(-1)[:-12].reshape(42,-1)[30:,:].reshape(-1)
    stimulus_code = stimulus_code.reshape(-1)[:-12].reshape(42,-1)[30:,:].reshape(-1)
    stimulus_type = stimulus_type.reshape(-1)[:-12].reshape(42,-1)[30:,:].reshape(-1)
    
    # Erase last 768 samples in order to reshape to (42,-1,64 )
    data = data.reshape(-1)[:-768].reshape(42,-1,64)
    data = data[30:,:,:] # Get first 20 characters for training data 
    data = data.reshape(-1, 64)
    fs = 240
    stimulus_code = stimulus_code[flashing == 1]
 
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
   # dat.sequence_phase = sequence_phase
    return dat


In [136]:
labels = 'CATDOGFISHWATERBOWLHATHATGLOVESHOESFISHRAT'
path_ = '../BCI_Comp_II_2003/data/'
datafiles = [join(path_,f) for f in listdir(path_) if isfile(join(path_, f))]
training_set = [f for i,f in enumerate(datafiles) if '12' not in f and 'eloc64' not in f]
testing_set = [f for i,f in enumerate(datafiles) if '12' in f and 'eloc64' not in f]

In [160]:
# load the training set
dat_train = load_bci_TRAINdata2(training_set)


In [161]:
TRUE_LABELS = labels[30:]
TRUE_LABELS

'SHOESFISHRAT'

### Training

In [162]:
def train_d2(dat):
    cnt = dat

    fs_n = cnt.fs / 2

    b, a = proc.signal.butter(5, [10 / fs_n], btype='low')
    cnt = proc.lfilter(cnt, b, a)

#     b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
#     cnt = proc.lfilter(cnt, b, a)

    cnt = proc.subsample(cnt, 20)

    epo = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL)

    #from wyrm import plot
    #plot.plot_spatio_temporal_r2_values(proc.sort_channels(epo))
    #print JUMPING_MEANS_IVALS
    #plot.plt.show()

    fv = proc.create_feature_vectors(epo)
    X = fv.data
    y = fv.axes[0]
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, train_size = 0.7)

    #     clf = proc.lda_train(fv)
    clf = LinearDiscriminantAnalysis(solver='eigen')
    clf.fit(X,y)
    return clf

In [163]:
clf = train_d2(dat_train)

In [164]:
subject = 'C'
filename = 'Trained Classifiers/'+subject+'/LDA_'+subject+'.pkl'

In [165]:
joblib.dump(clf, filename)

['Trained Classifiers/C/LDA_C.pkl']

### Online experiment

In [166]:
bbuffer = BlockBuffer(12)
dat_test = load_bci_TESTdata2(training_set)
cnt = dat_test

In [167]:
amp = RReeplayAmp()
amp.configure(data=cnt.data, marker=cnt.markers, channels=cnt.axes[-1], fs=cnt.fs,
              realtime=False, blocksize_samples=12)

In [168]:
online_experiment(amp, clf, subject)

DEBUG:__main__:Markers processed: 
DEBUG:__main__:0
DEBUG:__main__:Markers processed: 
DEBUG:__main__:1
DEBUG:__main__:Markers processed: 
DEBUG:__main__:2
C:\Users\aleja\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aleja\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aleja\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
DEBUG:__main__:Markers processed: 
DEBUG:__main__:3
DEBUG:__main__:Markers processed: 
DEBUG:__main__:4
DEBUG:__main__:Markers processed: 

Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 6.248712539672852
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 8.486509323120117
Predicho:  [1] 	Verdadero:  [0] [False]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  0.6666666666666666
time 11.019706726074219
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  0.75
time 9.973526000976562
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  0.8
time 9.972572326660156
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  0.8333333333333334
time 11.989355087280273
Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification 

DEBUG:__main__:Markers processed: 
DEBUG:__main__:10
DEBUG:__main__:Markers processed: 
DEBUG:__main__:11
DEBUG:__main__:Markers processed: 
DEBUG:__main__:12
DEBUG:__main__:Markers processed: 
DEBUG:__main__:13
DEBUG:__main__:Markers processed: 
DEBUG:__main__:14
DEBUG:__main__:Markers processed: 
DEBUG:__main__:15
DEBUG:__main__:Markers processed: 
DEBUG:__main__:16
DEBUG:__main__:Markers processed: 


Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  1.0 	f1:  0.8 	accuracy:  0.9090909090909091
time 11.415481567382812
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  1.0 	f1:  0.8 	accuracy:  0.9166666666666666
time 25.472402572631836
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  1.0 	f1:  0.8 	accuracy:  0.9230769230769231
time 18.075227737426758
Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8571428571428571
time 11.147260665893555
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8666666666666667
time 11.968612670898438
Predicho:  [0] 	Verdadero:  [0

DEBUG:__main__:17
DEBUG:__main__:Markers processed: 
DEBUG:__main__:18
DEBUG:__main__:Markers processed: 
DEBUG:__main__:19
DEBUG:__main__:Markers processed: 
DEBUG:__main__:20
DEBUG:__main__:Markers processed: 
DEBUG:__main__:21
DEBUG:__main__:Markers processed: 
DEBUG:__main__:22
DEBUG:__main__:Markers processed: 


...

Character prediction metrics...
Current accuracy: 90.0
10/12 characters predicted
time 20.449161529541016
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.7379518072289156 	precision:  0.5201698513800425 	f1:  0.6102117061021171 	accuracy:  0.8422379032258065
Character prediction metrics...
Current accuracy: 90.0
10/12 characters predicted
time 23.06509017944336
Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.7379518072289156 	precision:  0.5190677966101694 	f1:  0.6094527363184079 	accuracy:  0.8418136020151133
Character prediction metrics...
Current accuracy: 90.0
10/12 characters predicted
time 24.05071258544922
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.7379518072289156 	precision:  0.5190677966101694 	f1:  0.6094527363184079 	accuracy:  0.8418932527693856
Character prediction metrics...
Current accuracy: 90.0
10/12 characters predicted
time 22.855520248413086
Predicho:  [0]

DEBUG:__main__:SHOESFISHRAT
DEBUG:__main__:shoesfisnr
DEBUG:__main__:Markers processed: 
DEBUG:__main__:178
DEBUG:__main__:SHOESFISHRAT
DEBUG:__main__:shoesfisnr
DEBUG:__main__:Markers processed: 
DEBUG:__main__:179
DEBUG:__main__:SHOESFISHRAT
DEBUG:__main__:shoesfisnr


Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.7379518072289156 	precision:  0.5168776371308017 	f1:  0.607940446650124 	accuracy:  0.8410462776659959
Character prediction metrics...
Current accuracy: 90.0
10/12 characters predicted
time 39.101600646972656
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.7379518072289156 	precision:  0.5168776371308017 	f1:  0.607940446650124 	accuracy:  0.8411261940673705
Character prediction metrics...
Current accuracy: 90.0
10/12 characters predicted
time 36.810874938964844
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.7379518072289156 	precision:  0.5168776371308017 	f1:  0.607940446650124 	accuracy:  0.8412060301507538
Character prediction metrics...
Current accuracy: 90.0


DEBUG:__main__:Markers processed: 
DEBUG:__main__:180
DEBUG:__main__:SHOESFISHRAT
DEBUG:__main__:shoesfisnra


10/12 characters predicted
time 61.072349548339844
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.7379518072289156 	precision:  0.5168776371308017 	f1:  0.607940446650124 	accuracy:  0.8412857860371673
Character prediction metrics...
Current accuracy: 90.9090909090909
11/12 characters predicted
Accuracy: 90.9090909090909


In [14]:
a= list(np.arange(6))
b = [True, False, True, True, True, False]

In [15]:
a[~b]

TypeError: bad operand type for unary ~: 'list'

In [10]:
a

array([0, 1, 2, 3, 4, 5])